## 역전파(Back-propagation) 파헤치기 1

***(!!! 역전파를 제대로 이해하고 직접 구현해보기 위해 정리한 글이지만, 개인적인 호기심을 위해 해 본 프로젝트여서 구성이 다소 주관적입니다. 무엇보다도, 딥러닝 패키지를 사용하는 데엔 전혀 알 필요 없는 내용들입니다!!!)***

이번 포스팅 1, 2 에서는, 순방향 모델의 가장 단순한 형태인 다층 퍼셉트론(MLP) 모델의 역전파를 직접 구현해보고, 직접 구현한 루틴과 PyTorch의 내장 optimizer와 성능 및 속도를 비교해보는 것을 목표로 하였습니다.

## 배경

신경망 학습을 관통하는 가장 큰 방법론은 모델 output과 타겟을 비교한 손실값(Loss)을 경사하강법(Steepest Descent Method)을 이용해 최소화하는 방향으로 모델 파라미터를 업데이트 하는 것입니다. 보통 순방향 신경망에서는 '역전파'라고 하는데, 이를 제대로 이해하기 위해선 신경망의 전체 파라미터들 $\mathbf{P}$ 에 대한 손실함수의 미분이 어떤 구조를 가지는지 알아야 합니다. 

수식으로 써보면 다음과 같습니다: 모델 output $\mathbf{x}_\mathrm{out}$ 와 해당 타겟 $\mathbf{y}$ 에 대한 손실함수를 $\mathrm{Loss}(\mathbf{x}_\mathrm{out}\,;\,\mathbf{y})$ 라 하면,

$$ \frac{\partial}{\partial \mathbf{P}} \mathrm{Loss}(\mathbf{x}_\mathrm{out}(\mathbf{P})\,;\,\mathbf{y}) = \underbrace{\frac{\partial}{\partial \mathbf{x}_{\mathrm{out}}}\mathrm{Loss}(\mathbf{x}_\mathrm{out}\,;\,\mathbf{y})}_{(\mathrm{A})}\;\underbrace{\frac{\partial}{\partial \mathbf{P}}\mathbf{x}_\mathrm{out}}_{(\mathrm{B})} $$

가 되는데, $(\mathrm{A})$ 파트는 온전히 손실함수의 구조에 의해 결정이 되고, $(\mathrm{B})$ 파트는 온전히 신경망의 구조에 의해 결정이 됩니다. 즉, $(\mathrm{B})$ 파트의 미분을 이해하는 것이 신경망 학습에 핵심적인 역전파를 이해하는 것과 큰 관련이 있는데, *문제는 가장 간단한 모델인 MLP 모델만 하더라도 벡터 (바이어스) 뿐만이 아닌 가중치 행렬들을 파라미터로 가지고 있고, 모델에 따라 더 나아가서는 더욱 복잡한 형태의 파라미터들을 (무려 수십, 수백개의 레이어마다) 가지고 있을 수 있어서 이것이 간단한 일이 아니라는 것입니다.*

역전파의 원리는 흔히 다변수 미분법인 체인 룰(chain-rule)로 설명하곤 하는데, 체인 룰을 제대로 알기 위해선 벡터와 행렬의 개념을 확장한 **'텐서(Tensor)'** 개념과 텐서의 **'수축(Contraction)'** 이라는 연산법을 알아야 합니다. 체인 룰은 결국 텐서간의 수축으로 간편히 표현할 수 있게 됨을 알아보고, 그것을 이용하여 수많은 신경망의 변수들이 얽힌 체인 룰로 표현되는 역전파를 코딩으로 구현해 보겠습니다.


## 텐서의 정의

(공학적인 의미에서의) 텐서는, 기본적으로 *벡터와 행렬의 확장*이라고 생각하시면 편합니다. 즉, 벡터는 1차원 텐서, 행렬은 2차원 텐서(그리고 스칼라는 0차원 텐서) 로 생각할 수 있습니다. 텐서의 차원은 index 자유도의 갯수로 볼 수 있는데요, 즉 어떤 텐서 $\mathbf{T}$ 가 $K$ 차원이라면, $\mathbf{T}$ 의 각 원소는

$$(\mathbf{T})_{i_0,i_1,\dots,i_{K-1}},\quad 1\le i_k\le N_k $$

로 쓸 수 있고, '$K$차원 텐서'는 줄여서 '$K$-텐서'라고 부릅니다. $\mathbf{T}$는 총 $N_0\times N_1\times \cdots\times N_{K-1}$ 개의 원소를 갖는 (다차원)어레이로 정의할 수 있습니다.

쉬운 예제로, 어떤 $4\times 3$ 행렬 $\mathbf{M}$ 은, 각 원소를

$$ (\mathbf{M})_{i_1,i_2},\quad 1\le i_1\le 4,\; 1\le i_2\le 3 $$

으로 표기할 수 있는, 2개의 index 자유도를 갖는 2-텐서로 생각할 수 있습니다.

# 텐서의 수축

수축(Contraction)은, 두 텐서 사이의 연산이며 **같은 갯수의 원소를 갖는 차원끼리 점곱(dot product)를 실시하는 연산** 으로, 벡터들 사이의 점곱을 텐서에 확장한 개념으로 볼 수 있습니다. 먼저, *같은 길이의* 두 벡터 $\mathbf{a}=(\mathbf{a}_{i})$ 와 $\mathbf{b} = (\mathbf{b}_j)$ ($i,j=1,\dots,N$) 사이의 점곱은 엄밀하게 다음과 같이 쓸 수 있습니다.

\begin{align}
\mathbf{a}\cdot\mathbf{b} & = \sum_{i,j} \delta_{i,j}(\mathbf{a})_i(\mathbf{b})_j.
\end{align}

여기서, $\delta_{i,j}$ 는 Kronecker-delta ($i=j$ 이면 1을, $i\not=j$ 이면 0을 원소로 갖는 일종의 2-텐서) 표기입니다. 즉, 같은 인덱스의 원소끼리만 곱해준 뒤 각각을 더해준다는 의미입니다. 2중합 $\sum_{i,j}$는, Kronecker-delta의 영향으로 $i=j$ 일때만 고려하면 되므로 다음과 같이 더 친숙한 표현으로 바뀝니다.

$$\mathbf{a}\cdot\mathbf{b} = \sum_{i} (\mathbf{a})_i(\mathbf{b})_i. $$

이 벡터 점곱의 개념을 텐서 개념에 확장하면, 다음과 같이 쓸 수 있습니다: 

두 $m$-, $n$-텐서
$(\mathbf{T})_{i_0,i_1,\dots,i_{m-1}},\;\; (\mathbf{S})_{j_0,j_1,\dots,j_{n-1}}$ 에 대해, 각각의 index들을 집합표현으로 간단히 $I = \{i_0,i_1,\dots,i_{m-1}\}$, $J=\{j_0,j_1,\dots,j_{n-1}\}$ 로 나타냅시다. 서로 점곱을 해 줄 차원을 지정해주는 $d$ 개의 index 쌍들의 집합을 $\sigma=\{i_{m_k},j_{n_k}\}_{k=0}^{d-1} $ 로 나타낸다면, $\mathbf{T}$ 와 $\mathbf{S}$ 사이의 $\sigma$ index 끼리의 점곱에 의한 수축은 새로운 텐서를 만들고, 그것의 각 성분들은 구체적으로 다음과 같이 정의됩니다:

$$ (\mathbf{T} \bullet_{\sigma} \mathbf{S})_{(I\cup J)\setminus\sigma} = \sum_{i_{m_0},j_{n_0},\dots,i_{m_{d-1}},j_{m_{d-1}}} \delta_{i_{m_0},j_{n_0}}\delta_{i_{m_1},j_{n_1}}\cdots \delta_{i_{m_{d-1}},j_{m_{d-1}}}(\mathbf{T})_{i_0,i_1,\dots,i_{m-1}}(\mathbf{S})_{j_0,j_1,\dots,j_{n-1}}. \quad\quad\quad\cdots\;(*)$$

**(당연히, 점곱을 해 줄 index의 차원끼리는 원소의 갯수가 맞아야 합니다.)** 벡터(1차원 텐서)끼리의 점곱이 $1+1-2 = 0$ 에 의해 스칼라(0차원 텐서)를 반환하듯이, $m$-텐서와 $n$-텐서의 $d$ 개의 차원들에 대한 점곱($\sigma$ 집합으로 지시해준)에 의한 수축은 인덱스 집합으로 $(I\cup J)\setminus\sigma$ 를 갖는 새로운 $m+n-2d$ 차원의 텐서를 만듭니다. 

우리가 흔히 아는 행렬과 행렬의 곱셈도 결국 2차원 텐서끼리의 1개 차원의 수축으로 표현할 수 있습니다. 어떤 두 $M_0\times M_1$, $N_0\times N_1$ 행렬 $(\mathbf{A})_{i_0,i_1}$ 와 $(\mathbf{B})_{j_0,j_1}$ 의 행렬곱 $\mathbf{A}\mathbf{B}$는 $M_1 = N_0$ 일 때만 정의되며, 결과는 $M_0\times N_1$ 행렬인 것을 떠올려 보면,

\begin{align}
(\mathbf{A}\mathbf{B})_{i_0,j_1} & = \sum_{k}(\mathbf{A})_{i_0,k}(\mathbf{B})_{k,j_1} \\
& = \sum_{i_1,j_0}\delta_{i_1,j_0}(\mathbf{A})_{i_0,i_1}(\mathbf{B})_{j_0,j_1} \\
& = (\mathbf{A}\; \bullet_{\{i_1,j_0\}} \;\mathbf{B})_{i_0,j_1},
\end{align}

즉, $\mathbf{A}\mathbf{B} = \mathbf{A}\; \bullet_{\{i_1,j_0\}} \;\mathbf{B}$ 로써 행렬곱은 첫 번째 행렬의 행과 두 번째 행렬의 열 끼리 점곱을 실시하는, 위 $(*)$ 식의 특별한 경우임을 알 수 있습니다.

개념이 복잡하므로, 예제를 통해 익혀 보겠습니다. 먼저, Torch로 임의의 2-텐서를 만들어 보겠습니다.

In [1]:
import torch as t
t.manual_seed(1)

A = t.randn(4,4); print(f"A =\n{A}\n")
B = t.randn(4,5); print(f"B =\n{B}\n")

A =
tensor([[-1.5256, -0.7502, -0.6540, -1.6095],
        [-0.1002, -0.6092, -0.9798, -1.6091],
        [-0.7121,  0.3037, -0.7773, -0.2515],
        [-0.2223,  1.6871,  0.2284,  0.4676]])

B =
tensor([[-0.6970, -1.1608,  0.6995,  0.1991,  0.1991],
        [ 0.0457,  0.1530, -0.4757, -1.8821, -0.7765],
        [ 2.0242, -0.0865,  2.3571, -1.0373,  1.5748],
        [-0.6298,  2.4070,  0.2786,  0.2468,  1.1843]])



$\mathbf{A}$ 는 $4\times 4$ 개의 원소를, $\mathbf{B}$는 $4\times 5$개의 원소를 갖습니다. 이 두 2차원 텐서(행렬)의 행렬곱을 계산해 보면,

In [2]:
print(f"A * B = \n{t.matmul(A,B)}")

A * B = 
tensor([[ 0.7190, -2.1613, -2.7002,  1.3896, -2.6571],
        [-0.9278, -3.7652, -2.5380,  1.7459, -2.9955],
        [-0.9048,  0.3351, -2.5449,  0.0309, -1.8995],
        [ 0.3999,  1.6219, -0.2893, -3.3412, -0.4408]])


로 당연히 $4\times 5$ 행렬이 됩니다. 위에서 두 텐서의 수축의 특별한 경우가 행렬곱임을 보았으므로, Torch가 제공하는 *torch.tensordot()* 내장함수를 사용해 행렬곱을 텐서간의 수축으로 표현해 보겠습니다. $\mathbf{A}$의 1 번째 차원과 $\mathbf{B}$의 0 번째 차원을 점곱하는 $\mathbf{A}\; \bullet_{\{i_1,j_0\}} \;\mathbf{B}$ 이 결국 행렬곱이므로, torch.tensordot() 함수도 점곱할 index $i_1,j_0$ 쌍을 비슷한 문법을 사용해 인수로 받습니다.

In [3]:
print(f"A cont[[1],[0]] B = \n{t.tensordot(A,B,dims=[[1],[0]])}\n")

A cont[[1],[0]] B = 
tensor([[ 0.7190, -2.1613, -2.7002,  1.3896, -2.6571],
        [-0.9278, -3.7652, -2.5380,  1.7459, -2.9955],
        [-0.9048,  0.3351, -2.5449,  0.0309, -1.8995],
        [ 0.3999,  1.6219, -0.2893, -3.3412, -0.4408]])



(위에서 인수 dims=[[1],[0]]) 인수가 바로 $\sigma = \{i_1,j_0\}$를 뜻합니다) 즉, 위의 행렬곱 $\mathbf{A}\mathbf{B}$ 같은 결과임을 확인할 수 있습니다.

그런데, tensordot의 인수를 dims=[[1],[0]] 에서 dims=[[0],[0]] 으로 바꾸면 어떻게 될까요? 이 경우는 $\sigma = \{i_0,j_0\}$가 되는 상황인데, 두 행렬의 열끼리 점곱을 실시하는 것과 같으므로, $\mathbf{A}^\top\mathbf{B}$ 가 됩니다.

In [4]:
print(f"A cont[[0],[0]] B = \n{t.tensordot(A,B,dims=[[0],[0]])}\n")
print(f"A^T * B = \n{t.matmul(t.transpose(A,0,1),B)}\n")

A cont[[0],[0]] B = 
tensor([[-0.2428,  1.2822, -2.7601,  0.5687, -1.6106],
        [ 0.0472,  4.8122,  0.9508,  1.0985,  2.8001],
        [-1.3063,  1.2263, -1.7600,  2.5766, -0.3229],
        [ 0.2447,  2.7695, -0.8229,  3.0844,  1.0870]])

A^T * B = 
tensor([[-0.2428,  1.2822, -2.7601,  0.5687, -1.6106],
        [ 0.0472,  4.8122,  0.9508,  1.0985,  2.8001],
        [-1.3063,  1.2263, -1.7600,  2.5766, -0.3229],
        [ 0.2447,  2.7695, -0.8229,  3.0844,  1.0870]])



$\mathbf{A}\; \bullet_{\{i_0,j_0\}} \;\mathbf{B}$ 는 $\mathbf{A}^\top\mathbf{B}$ 와 결과가 같음을 알 수 있습니다 (같은 원리로, $\mathbf{A}\mathbf{B}^\top$, $\mathbf{A}^\top\mathbf{B}^\top$ 등도 '차원이 맞다면' 표현 가능합니다).

마지막 연습으로, $5\times 4\times 7\times 3$ 텐서와 $2\times 5\times 7\times 4$ 텐서의 차원 (0,1), (1,3), (2,2) 끼리의 수축은 어떤 텐서가 될까요?

4차원 텐서들끼리의 3차원간 수축이므로, $4+4-2\times 3 = 2$차원 텐서가 됩니다. 이를 구현해보면,

In [5]:
A = t.randn(5,4,7,3); B = t.randn(2,5,7,4)
print(f"A cont[[0,1,2],[1,3,2]] B =\n\n{t.tensordot(A,B,dims=[[0,1,2],[1,3,2]])}\n")

A cont[[0,1,2],[1,3,2]] B =

tensor([[ -2.8911,  -1.7296],
        [  1.1046,  -7.7448],
        [ -3.1442, -10.0571]])



즉, 예상대로 $3\times 2$ 크기의 2차원 텐서가 나오는 것을 확인할 수 있습니다.

## 도함수와 텐서

이제, 일반적인 형태의 함수들의 도함수들은 텐서 표기를 활용해 간단히 표현할 수 있음을 알아보겠습니다. 

예를 들어, 어떤 다중함수 $\mathbf{F}:\mathbb{R}^3\to\mathbb{R}^2$ (즉, 벡터를 인수로 받아 벡터를 내놓는 벡터장(vector field)입니다) 

$$ \mathbf{F}(\mathbf{x}) =  \begin{bmatrix} 
F_1(x_1,x_2,x_3) \\
F_2(x_1,x_2,x_3)
\end{bmatrix}, \quad\quad F_i:\mathbb{R}^3\to\mathbb{R},\;\;x_j\in\mathbb{R}
$$

가 있다고 합시다 (즉, $\mathbf{x}=(x_1,x_2,x_3)$). 이것의 도함수(Jacobian)는 $\frac{\partial \mathbf{F}}{\partial\mathbf{x}}$ 로 표기하며 **가능한 모든 편미분값 $\frac{\partial F_i}{\partial x_j}$ 를 원소로 갖는 텐서**로 생각할 수 있습니다. 다시 말해서, $\mathbf{F}$ 와 $\mathbf{x}$ 는 각각 1차원씩의 자유도를 갖는(흔히 말하는 벡터의 유클리드 차원 개념과는 다릅니다!) 1-텐서(벡터)이고, 그것의 도함수는 해당 1-텐서 두개의 가능한 모든 원소들의 조합을 가지므로, **다음과 같은 6개 원소를 갖는 2-텐서로 생각할 수 있습니다:**

$$ \left(\frac{\partial \mathbf{F}}{\partial\mathbf{x}}\right)_{i,j} = \frac{\partial F_i}{\partial x_j}, \qquad(i=1,2,\;j = 1,2,3) .$$

하나 더 추가할 점은, 이제부터는 미분을 하는 변수의 index는 아래 첨자로, 미분이 되는 변수(함수)의 index는 위 첨자로 

$$ \left(\frac{\partial \mathbf{F}}{\partial\mathbf{x}}\right)^{\color{red}i}_{\color{red}j} = \frac{\partial F^{\color{red}i}}{\partial x_{\color{red}j}}$$

와 같이 쓸 것입니다 (물리학에서는 해당 텐서의 위 첨자에 표시한 차원은 '반변(contravariant)성분', 아래 첨자에 표시한 차원은 '공변(covariant)성분'이라고 부릅니다. 이는 곧 언급할 체인 룰의 계산에 필요한 **아인슈타인 합의 규약(Einstein Summation Convention)** 이라는 특수한 형태의 텐서 수축 표기와 관련이 있습니다.)

이제, 더 나아가 어떤 $n$-텐서 $\mathbf{S}$ 를 변수로 갖는 $m$-텐서함수 $\mathbf{T}(\mathbf{S})$ 를 생각해봅시다. 그렇다면 역시 $\mathbf{T}$ 의 $\mathbf{S}$ 에 대한 도함수도 정의할 수 있는데, 그것은 $\mathbf{T}$ 의 모든 원소들을 $\mathbf{S}$ 의 모든 원소들로 편미분한 값들을 원소로 가져야 하므로 $(m+n)$-텐서가 되고 그것의 각 원소들은

$$ \left(\frac{\partial}{\partial\mathbf{S}} \mathbf{T}(\mathbf{S})\right)^{i_1,\dots,i_m}_{j_1,\dots,j_n} = \frac{\partial (\mathbf{T})^{i_1,\dots,i_m}}{\partial \bf({S})_{j_1,\dots,j_n}} $$

로 쓸 수 있습니다.

## 체인 룰과 아인슈타인 표기

이제, $\mathbf{T}$ 를 미분했던 텐서 $\mathbf{S}$ 가 다른 $l$-텐서 $(\mathbf{U})_{k_1,\dots,k_l}$ 에 대한 함수였다고 가정해봅시다. 그러면, $\mathbf{T}$ 의 $\mathbf{U}$ 에 대한 도함수 텐서는, $(m+l)$-텐서이고, 다음과 같이 표현됩니다:

$$ \left(\frac{\partial}{\partial\mathbf{U}}\mathbf{T}(\mathbf{S}(\mathbf{U}))\right)^{i_1,\dots,i_m}_{k_1,\dots,k_l} = \sum_{\color{red}j_1,\dots,j_n}\frac{\partial (\mathbf{T})^{i_1,\dots,i_m}}{\partial (\mathbf{S})_{\color{red}j_1,\dots,j_n}}\;\frac{\partial (\mathbf{S})^{\color{red}j_1,\dots,j_n}}{\partial (\mathbf{U})_{k_1,\dots,k_l}}. \qquad\qquad\cdots \;(**)$$

**이것이 일반적인 형태의 체인 룰이며, 두 도함수 텐서 $\frac{\partial\mathbf{T}}{\partial\mathbf{S}}$ 와 $\frac{\partial\mathbf{S}}{\partial\mathbf{U}}$ 의 $\mathbf{S}$ 의 성분에 해당하는 index 끼리의 수축**입니다!

체인 룰은 '아인슈타인 합의 규약'을 이용하면 더 간단히 표현할 수 있습니다. 아인슈타인 합의 규약은, 간단히 말해서 어떤 두 텐서 간의 성분의 곱을 표현할 때, *'같은 인덱스가 위, 아래 첨자에 반복되면 그 차원에 대해 점곱을 실시한 것으로 간주한다'* 로 요약할 수 있습니다. 즉, 예를 들어

$$ (\mathbf{T})^{i_0}_{i_1,{\color{red}k}}(\mathbf{S})^{{\color{red}k}}_{j_1} $$

와 같이 썼다면 자연히 그 표기는

\begin{align}
(\mathbf{T})^{i_0}_{{\color{red}k},i_2}(\mathbf{S})^{{\color{red}k}}_{j_1} & = \sum_{\color{red}i_1,j_0}\delta_{\color{red}i_1,j_0} (\mathbf{T})^{i_0}_{{\color{red}i_1},i_2}(\mathbf{S})^{\color{red}j_0}_{j_1} \\ 
& = (\mathbf{T}\;\bullet_{\{{\color{red}i_1,j_0}\}}\;\mathbf{S})^{i_0}_{i_2,j_1},
\end{align}

즉 $\mathbf{T}$ 와 $\mathbf{S}$의 해당 차원의 점곱을 통한 수축으로 나온 텐서의 $i_0,i_2,j_1$ 번째 성분임을 의미하는 것입니다.

따라서 체인 룰 $(**)$ 는 다음과 같이 더 간결하게 쓸 수 있습니다:

$$ \left(\frac{\partial }{\partial\mathbf{U}}\mathbf{T}(\mathbf{S}(\mathbf{U}))\right)^{i_1,\dots,i_m}_{k_1,\dots,k_l} = \frac{\partial (\mathbf{T})^{i_1,\dots,i_m}}{\partial (\mathbf{S})_{j_1,\dots,j_n}}\;\frac{\partial (\mathbf{S})^{j_1,\dots,j_n}}{\partial (\mathbf{U})_{k_1,\dots,k_l}}. $$

이제 다시 쉬운 예로, 앞서 정의한 벡터장 함수 $\mathbf{F}$ 의 각 변수들, $x_1$ 그리고 $x_2$, $x_3$ 가 다시 $t$ 라는 스칼라 변수에 대한 함수였다고 가정해 봅시다. 그러면 체인 룰에 의해, 1-텐서 $\mathbf{F}$ 를 0-텐서 $t$에 대해 미분한 결과는 $1+0=1$ 텐서이며, 그 원소들을 $i$ 로 인덱싱 하면 아인슈타인 표기에 의해:

$$
\left(\frac{\partial}{\partial t} \mathbf{F}(\mathbf{x}(t))\right)^i =  \left(\frac{\partial}{\partial \mathbf{x}} \mathbf{F}(\mathbf{x})\right)^i_{\color{red}j}\;\left(\frac{\partial}{\partial t}\mathbf{x}(t)\right)^{\color{red}j} \\
\!\!\!\!= \frac{\partial F_i}{\partial x_{\color{red}j}} \frac{\partial x_{\color{red}j}}{\partial t} \\
\qquad\qquad\qquad\qquad\qquad\qquad\,\;\;\;\;\;= \frac{\partial F_i}{\partial x_1}\frac{\partial x_1}{\partial t} + \frac{\partial F_i}{\partial x_2}\frac{\partial x_2}{\partial t}+ \frac{\partial F_i}{\partial x_3}\frac{\partial x_3}{\partial t}, \quad i = 1,2
$$

가 됩니다. 이는 $\mathbf{F}$ 의 각 성분별로 기본적인 체인 룰을 적용한 결과와 같음을 알 수 있습니다.

## MLP와 체인 룰

이제 모든 준비가 끝났습니다. 본격적으로 다음 포스팅에서 MLP의 역전파에 대해 다루기 전, 연습삼아 가중치행렬 $\mathbf{W}$ 와 바이어스 벡터 $\mathbf{b}$ 를 파라미터로 갖고 시그모이드 함수 $\Phi(\mathbf{x}) = [\phi(x_1)\;\phi(x_2\;)\;\dots\;\;]$ 를 activation으로 갖는 하나의 퍼셉트론 변환

$$ \mathcal{L}(\mathbf{x};\mathbf{W},\mathbf{b}) = \Phi(\mathbf{W}\mathbf{x}+\mathbf{b}) $$

을 가중치 행렬 파라미터 $\mathbf{W}$ 에 대해 미분을 해 보겠습니다. 이 레이어 함수는 다음 레이어로 넘길 다른 벡터를 반환하는 벡터(1-텐서)함수이고, $\mathbf{W}$ 는 2-텐서 이므로 이 도함수 $\frac{\partial}{\partial \mathbf{W}}\mathcal{L}(\mathbf{x};\mathbf{W},\mathbf{b})$는 $1+2=3$-텐서가 됩니다. 여기서, $\mathbf{W}\mathbf{x}+\mathbf{b} = \mathbf{h}$ 로 표기하면, 아인슈타인 표기를 적용한 체인 룰에 의해

$$ \left(\frac{\partial}{\partial\mathbf{W}}\mathcal{L}(\mathbf{x};\mathbf{W},\mathbf{b})\right)^i_{j,k}= \left(\frac{\partial}{\partial\mathbf{h}}\Phi(\mathbf{h})\right)^i_{\color{red}l}\;\left(\frac{\partial}{\partial \mathbf{W}}(\mathbf{W}\mathbf{x}+\mathbf{b})\right)^{\color{red}l}_{j,k} $$

로 쓸 수 있고, 각각의 도함수 텐서는 세부적으로 다음과 같이 계산됩니다:

\begin{align}
\left(\frac{\partial}{\partial\mathbf{h}}\Phi(\mathbf{h})\right)^i_{l} & = \frac{\partial}{\partial h^l} \phi(h^i) = \phi'(h^i)\delta^i_l, \\
\left(\frac{\partial}{\partial \mathbf{W}}(\mathbf{W}\mathbf{x}+\mathbf{b})\right)^{l}_{j,k} & = \frac{\partial}{\partial W_{j,k}} (W^l_n x^n + b^l) = \delta^l_j\delta_{k,n}x^n = \delta^l_j x_k \;\;(\;\because \;\delta_{k,n}x^n=x_k).
\end{align}

따라서, 

\begin{align}
\left(\frac{\partial}{\partial\mathbf{W}}\mathcal{L}(\mathbf{x};\mathbf{W},\mathbf{b})\right)^i_{j,k} & = \phi'(h^i)x_k\delta^i_l\delta^l_j \\
& = \phi'(h^i) x_k \delta^i_j
\end{align}

이 됩니다. 이 때, sigmoid 함수의 도함수 $\phi'(x)$ 는 $\phi(x)(1-\phi(x))$ 임이 잘 알려져 있으므로, 더 간단히는

$$ = \phi(h^i)(1-\phi(h^i))x_k\delta^i_j  $$

로 쓸 수 있습니다.

## 결론

위와 같이 각 레이어 변환 별 파라미터의 미분을 forward message passing 도중 바로 구할 수 있는 텐서 형태로 나타낼 수 있음을 확인했습니다. 역전파에 필요한 도함수 텐서들은 다음 종류들인데,

\begin{align}
\left(\frac{\partial}{\partial\mathbf{h}}\Phi(\mathbf{h})\right)^i_j & = \phi(h^i)(1-\phi(h^i))\delta^i_j \\
\left(\frac{\partial\mathbf{h}}{\partial\mathbf{x}}\right)^i_j & = W^i_j \\
\left(\frac{\partial\mathbf{h}}{\partial\mathbf{W}}\right)^i_{j,k} & = \delta^i_j x_k \\
\left(\frac{\partial\mathbf{h}}{\partial\mathbf{b}}\right)^i_j & = \delta^i_j
\end{align}

forward 도중 필요한 정보들을 받아 위 도함수 텐서들을 구현하는 루틴들을 아래와 같이 만든 뒤

In [ ]:
def get_dL_dh(h):
  dphi_dh = t.zeros(h.shape[0],h.shape[0],h.shape[1])
  for i in range(dphi_dh.shape[0]):
    for k in range(dphi_dh.shape[2]):
      phiprime = t.sigmoid(h[:,k]) * (t.ones(h.shape[0]) - t.sigmoid(h[:,k]))
      dphi_dh[i,i,k] = phiprime[i]
  return dphi_dh

def get_dh_dx(h,x,W):
  dh_dx = t.zeros(h.shape[0],x.shape[0],x.shape[1])
  for k in range(dh_dx.shape[2]):
    dh_dx[:,:,k] = W
  return dh_dx
  
def get_dh_dW(h,x,W):
  dh_dW = t.zeros(h.shape[0],W.shape[0],W.shape[1],x.shape[1])
  for i in range(dh_dW.shape[0]):
    for k in range(dh_dW.shape[3]):
      dh_dW[i,i,:,k] = x[:,k]
  return dh_dW

def get_dh_db(h,x,W):
  dh_db = t.zeros(h.shape[0],h.shape[0],x.shape[1])
  for k in range(dh_db.shape[2]):
    dh_db[:,:,k] = t.eye(h.shape[0])
  return dh_db

torch.tensordot()으로 체인 룰 연산을 적용해 역전파를 구현하는 것은 본격적으로 다음 글에 다뤄보겠습니다.